# Procesamiento de datos Censo INEGI 2020

Este notebook tiene como propósito demostrar el flujo de procesamiento de datos del Censo INEGI 2020.

In [2]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [3]:
from procesador_censo_2020 import *
from gestor_carga_datos import *

## Importación y preprocesamiento de datos

Se utiliza la función `preprocesar_conjunto_datos` para preprocesar los datos originales, el dataframe obtenido se guardará en la ruta especificada en el parámetro `destino` en caso de ser especificada.

Por otro lado, la función `importar_diccionario_datos` devuelve un objeto `dict` a partir del archivo `.csv` del diccionario original, por lo que no se guarda el resultado.

In [3]:
# diccionario = importar_diccionario_datos("../data/raw/iter_00_cpv2020/diccionario_datos/diccionario_datos_iter_00CSV20.csv")

# df_state = preprocesar_conjunto_datos("../data/raw/iter_00_cpv2020/conjunto_de_datos/conjunto_de_datos_iter_00CSV20.csv", escala="state", destino="../data/preprocessed/state_cpv2020.csv")
# df_mun = preprocesar_conjunto_datos("../data/raw/iter_00_cpv2020/conjunto_de_datos/conjunto_de_datos_iter_00CSV20.csv", escala="mun", destino="../data/preprocessed/mun_cpv2020.csv")
# df_ageb = preprocesar_conjunto_datos("../data/raw/ageb_mza_urbana", escala="ageb", destino="../data/preprocessed/ageb_cpv2020.csv")

Al ser una ejecución lenta, en este caso se omite el uso de la función `preprocesar_conjunto_datos` y se importan los archivos `.csv` guardados previamente con la función `importar_conjunto_datos`.

In [4]:
diccionario = importar_diccionario_datos("../data/raw/iter_00_cpv2020/diccionario_datos/diccionario_datos_iter_00CSV20.csv")

df_state = importar_conjunto_datos("../data/preprocessed/state_cpv2020.csv", escala="state")
df_mun = importar_conjunto_datos("../data/preprocessed/mun_cpv2020.csv", escala="mun")
df_ageb = importar_conjunto_datos("../data/preprocessed/ageb_cpv2020.csv", escala="ageb")

## Procesamiento de datos

In [5]:
procesador = ProcesadorCenso2020(diccionario, df_state, df_mun, df_ageb)

Variables excluidas: {'NOM_MUN', 'LOC', 'MZA', 'ENTIDAD', 'LATITUD', 'NOM_LOC', 'NOM_ENT', 'AGEB', 'TAMLOC', 'ALTITUD', 'LONGITUD', 'MUN'}


### Procesamiento de una variable

Descripción: Se procesa la variable "POBFEM" (población femenina) a escala AGEB ("ageb"); se utiliza la variable "POBTOT" (población total) como base para calcular porcentaje, por lo que el resultado será procesar el porcentaje de población femenina con respecto a la población total; finalmente se especifica una clasificación en `q=10` categorías (deciles).

***Importante***: es posible que al especificar un número `q` de categorías, en el procesamiento se obtenga una categoría adicional, esto es debido a que en los conjuntos de datos existen entidades que, para ciertas variables, no tienen asginado un valor el cual pueda ser categorizado, todas estas entidades pertenecerán finalmente a una categoría "Sin clasificar".

In [6]:
procesar_una_variable = procesador.procesar_variable(escalas=["ageb"], var="POBFEM", base_porcentaje="POBTOT", q=10)
procesar_una_variable

,name,code,bin,interval_state,interval_mun,interval_ageb,cells_state,cells_mun,cells_ageb
0,Población femenina,POBFEM,1,<NA>,<NA>,-0.1%:47.7%,<NA>,<NA>,"{0100100011481,0100100013488,0100100013524,010..."
1,Población femenina,POBFEM,2,<NA>,<NA>,47.7%:49.4%,<NA>,<NA>,"{0100100010163,0100100010229,0100100011195,010..."
2,Población femenina,POBFEM,3,<NA>,<NA>,49.4%:50.1%,<NA>,<NA>,"{0100100010182,0100100010892,0100100010996,010..."
3,Población femenina,POBFEM,4,<NA>,<NA>,50.1%:50.8%,<NA>,<NA>,"{010010001006A,0100100010106,0100100010854,010..."
4,Población femenina,POBFEM,5,<NA>,<NA>,50.8%:51.3%,<NA>,<NA>,"{0100100010017,0100100010356,0100100010426,010..."
5,Población femenina,POBFEM,6,<NA>,<NA>,51.3%:51.8%,<NA>,<NA>,"{0100100010286,0100100010341,0100100010445,010..."
6,Población femenina,POBFEM,7,<NA>,<NA>,51.8%:52.4%,<NA>,<NA>,"{0100100010233,0100100010549,0100100010587,010..."
7,Población femenina,POBFEM,8,<NA>,<NA>,52.4%:53.1%,<NA>,<NA>,"{0100100010322,0100100010337,010010001045A,010..."
8,Población femenina,POBFEM,9,<NA>,<NA>,53.1%:54.3%,<NA>,<NA>,"{010010001038A,0100100010411,0100100010430,010..."
9,Población femenina,POBFEM,10,<NA>,<NA>,54.3%:100.0%,<NA>,<NA>,"{0100100010290,0100100010303,0100100010360,010..."


### Procesamiento de múltiples variables utilizando listas

Descripción: Para procesar múltiples variables se utiliza un diccionario, en donde la llave será la variable que se utilizará como base para calcular porcentaje, y el valor una lista de variables que se procesarán con esa base, en este caso:

- Se procesan los porcentajes de "PE_INAC_M", "PDER_ISTEE" y "P15SEC_COM" con respecto a "POBTOT"
- Se procesan los porcentajes de "VPH_AUTOM" y "VPH_EXCSA" con respecto a "VIVTOT"
- Se procesan las variables "REL_H_M", "PROM_HNV", "P_55A59_M" (sin calcular porcentajes con respecto a otra variable)

Lo anterior es a escala estatal ("state") y municipal ("mun"); finalmente se especifica una clasificación en `q=4` categorías (cuartiles).

In [9]:
dicc_listas = {
    "POBTOT" : ["PE_INAC_M", "PDER_ISTEE", "P15SEC_COM"],
    "VIVTOT" : ["VPH_AUTOM", "VPH_EXCSA"],
    None : ["REL_H_M", "PROM_HNV", "P_55A59_M"]
}

procesar_multi_metodo_listas = procesador.procesar_multiples_variables(escalas=["state", "mun"], dicc=dicc_listas, q=4)

Se muestra el procesamiento de los porcentajes con respecto a "POBTOT":

In [10]:
procesar_multi_metodo_listas["POBTOT"]

,name,code,bin,interval_state,interval_mun,interval_ageb,cells_state,cells_mun,cells_ageb
0,Población masculina de 12 años y más no económ...,PE_INAC_M,1,7.3%:8.6%,0.4%:8.0%,<NA>,"{04,07,11,14,16,21,23,27}","{01005,01011,02006,03008,04004,04006,04009,040...",<NA>
1,Población masculina de 12 años y más no económ...,PE_INAC_M,2,8.6%:9.1%,8.0%:9.4%,<NA>,"{01,03,06,12,13,22,29,30}","{01001,01002,01003,01004,01006,01007,01009,020...",<NA>
2,Población masculina de 12 años y más no económ...,PE_INAC_M,3,9.1%:9.7%,9.4%:11.2%,<NA>,"{02,15,17,18,24,28,31,32}","{01008,01010,02001,02002,02003,02005,03001,030...",<NA>
3,Población masculina de 12 años y más no económ...,PE_INAC_M,4,9.7%:10.4%,11.2%:28.7%,<NA>,"{05,08,09,10,19,20,25,26}","{05001,05003,05005,05008,05012,05013,05016,050...",<NA>
4,Población afiliada a servicios de salud en el ...,PDER_ISTEE,1,0.0%:0.2%,-0.1%:0.1%,<NA>,"{05,06,09,14,17,22,30,31}","{05001,05011,05014,05015,05019,05021,05023,050...",<NA>
5,Población afiliada a servicios de salud en el ...,PDER_ISTEE,2,0.2%:0.3%,0.1%:0.2%,<NA>,"{01,03,04,11,16,20,24,32}","{01005,01010,03001,03002,03008,04003,04006,040...",<NA>
6,Población afiliada a servicios de salud en el ...,PDER_ISTEE,3,0.3%:0.6%,0.2%:0.5%,<NA>,"{10,12,13,18,23,25,28,29}","{01001,01003,01006,01007,01009,01011,03009,040...",<NA>
7,Población afiliada a servicios de salud en el ...,PDER_ISTEE,4,0.6%:4.4%,0.5%:26.3%,<NA>,"{02,07,08,15,19,21,26,27}","{01002,01004,01008,02001,02002,02003,02004,020...",<NA>
8,Población masculina de 15 años y más con secun...,P15SEC_COM,1,6.3%:8.3%,0.6%:7.2%,<NA>,"{07,09,12,16,20,21,25,30}","{07001,07002,07003,07004,07006,07007,07008,070...",<NA>
9,Población masculina de 15 años y más con secun...,P15SEC_COM,2,8.3%:9.1%,7.2%:8.5%,<NA>,"{03,04,06,14,18,27,28,31}","{02006,03001,03003,04001,04002,04003,04005,050...",<NA>


### Cargar datos múltiples variables regex

Descripción: Otro método para procesar múltiples variables es utilizando un diccionario, en donde la llave será la variable que se utilizará como base para calcular porcentaje, y el valor una expresión regular, en este caso:

- Se procesan los porcentajes de las variables que conicidan con la expresión regular `"^POB(?!.\*TOT).\*"` con respecto a "POBTOT"
- Se procesan los porcentajes de las variables que conicidan con la expresión regular `"^VIV(?!.\*TOT).\*"` con respecto a "VIVTOT"
- Se procesan las variables que conicidan con la expresión regular `"^HOG"` (sin calcular porcentajes con respecto a otra variable)

Lo anterior es a escala estatal ("state"), municipal ("mun") y AGEB ("ageb"); finalmente se especifica una clasificación en `q=5` categorías (quintiles).

In [11]:
dicc_regex = {
    "POBTOT" : "^POB(?!.*TOT).*",
    "VIVTOT" : "^VIV(?!.*TOT).*",
    None : "^HOG"
}

procesar_multi_metodo_regex = procesador.procesar_multiples_variables_regex(escalas=["state", "mun", "ageb"], dicc=dicc_regex, q=5)

Se muestra el procesamiento de los porcentajes con respecto a "VIVTOT":

In [12]:
procesar_multi_metodo_regex["VIVTOT"]

,name,code,bin,interval_state,interval_mun,interval_ageb,cells_state,cells_mun,cells_ageb
0,Viviendas particulares de uso temporal,VIVPAR_UT,1,2.3%:4.4%,0.5%:4.5%,-0.1%:1.1%,"{01,02,05,09,15,27,28}","{01001,01002,01006,01009,01010,01011,02002,020...","{0100100010017,010010001045A,0100100010676,010..."
1,Viviendas particulares de uso temporal,VIVPAR_UT,2,4.4%:5.0%,4.5%:6.6%,1.1%:2.6%,"{08,11,19,24,25,30}","{01004,01005,01007,02006,03001,04003,04004,040...","{0100100010163,0100100010303,0100100010356,010..."
2,Viviendas particulares de uso temporal,VIVPAR_UT,3,5.0%:5.9%,6.6%:9.3%,2.6%:4.7%,"{04,06,10,22,26,29}","{01003,02001,03002,03003,03008,04001,04006,040...","{010010001006A,0100100010106,0100100010182,010..."
3,Viviendas particulares de uso temporal,VIVPAR_UT,4,5.9%:7.6%,9.3%:14.9%,4.7%:8.6%,"{07,14,16,21,23,32}","{01008,02003,02005,04007,05007,05008,05013,050...","{0100100010407,0100100010500,0100100010568,010..."
4,Viviendas particulares de uso temporal,VIVPAR_UT,5,7.6%:14.7%,14.9%:69.2%,8.6%:98.6%,"{03,12,13,17,18,20,31}","{03009,05001,05004,05005,05011,05012,05014,050...","{0100100010695,0100100011477,0100100014039,010..."
5,Viviendas particulares de uso temporal,VIVPAR_UT,6,<NA>,<NA>,Sin clasificar,<NA>,<NA>,"{0100100010229,0100100011782,0100100012992,010..."
6,Viviendas particulares habitadas,VIVPAR_HAB,1,65.3%:72.5%,7.4%:62.8%,-0.1%:66.7%,"{12,13,16,17,20,23,32}","{02005,05001,05004,05005,05012,05013,05014,050...","{0100100010017,0100100010638,0100100011034,010..."
7,Viviendas particulares habitadas,VIVPAR_HAB,2,72.5%:75.7%,62.8%:70.3%,66.7%:75.0%,"{03,06,10,14,18,31}","{01008,02003,03002,03009,05007,05008,05011,050...","{010010001006A,0100100010515,0100100010572,010..."
8,Viviendas particulares habitadas,VIVPAR_HAB,3,75.7%:77.0%,70.3%:75.4%,75.0%:80.5%,"{19,21,22,24,28,30}","{01003,03003,04007,04009,04011,05024,06001,060...","{0100100010106,0100100010163,0100100010182,010..."
9,Viviendas particulares habitadas,VIVPAR_HAB,4,77.0%:78.8%,75.4%:79.4%,80.5%:85.6%,"{04,07,08,11,26,29}","{01005,01010,01011,02001,02006,03008,04001,040...","{0100100010233,0100100010290,0100100010303,010..."


## Conexión a la base de datos

### Procesamiento

Se realiza un procesamiento de todas las variables (mediante una expresión regular que acepta cualquier cadena) a escala estatal ("state"), municipal ("mun") y AGEB ("ageb"); se especifica una clasificación en `q=10` categorías (deciles).

In [6]:
dicc_procesamiento_completo = {
    None : ".*"
}

procesamiento_completo = procesador.procesar_multiples_variables_regex(escalas=["state", "mun", "ageb"], dicc=dicc_procesamiento_completo, q=10)

La variable 'P_45A49_F' (var) no existe en el DataFrame de la escala 'ageb'
La variable 'P_0A4_F' (var) no existe en el DataFrame de la escala 'ageb'
La expresión regular '.*' coincide con una variable ('ENTIDAD') que no tiene valores numéricos en los DataFrames de las escalas especificadas, excluyendo
La variable 'P_25A29_F' (var) no existe en el DataFrame de la escala 'ageb'
La variable 'P_65A69_M' (var) no existe en el DataFrame de la escala 'ageb'
La variable 'P_65A69_F' (var) no existe en el DataFrame de la escala 'ageb'
La variable 'P_85YMAS' (var) no existe en el DataFrame de la escala 'ageb'
La variable 'P_15A19' (var) no existe en el DataFrame de la escala 'ageb'
La variable 'P_20A24_F' (var) no existe en el DataFrame de la escala 'ageb'
La variable 'P_30A34_F' (var) no existe en el DataFrame de la escala 'ageb'
La variable 'P_40A44_M' (var) no existe en el DataFrame de la escala 'ageb'
La variable 'P_20A24' (var) no existe en el DataFrame de la escala 'ageb'
La variable 'P_50

### Carga a la base de datos

In [8]:
gestor = GestorCargaDatos(env_path="../.env")

Conexión verificada exitosamente


In [12]:
gestor.cargar_dataframe_a_tabla(procesamiento_completo[None], "variables")

Datos insertados exitosamente en la tabla 'variables'
